In [1]:
from gurobipy import *
import csv
import pandas as pd

# Optimierung für 1 isoliertes Produkt

In [1]:
#Read data
df = pd.read_csv('Prophet_Brezeln.csv')

In [3]:
scenarios = list(df['Sc'])
demand = dict(zip(df.Sc, df['18074']))
prob = dict(zip(df.Sc, df.prob))

In [4]:
m = Model()

#Preise & Kosten (anonymisiert)
price = 1
costs = 1

#Variablen einführen
order = m.addVar(vtype=GRB.INTEGER, name = "order")
sales = m.addVars(scenarios, name = "sales")

Academic license - for non-commercial use only


In [7]:
#Sales & Demand Nebenbedingung
m.addConstrs((sales[s] <= order for s in scenarios), name = "Cons1")
m.addConstrs((sales[s] <= demand[s] for s in scenarios), name = "Cons2")

In [10]:
obj = quicksum(  (prob[s]*sales[s]*price) for s in scenarios ) - order * costs 


m.setObjective(obj, GRB.MAXIMIZE)


m.optimize()
m.printAttr('X')

# Optimierung mit Berücksichtigung von Substitution

In [ ]:
#Daten einlesen
df_p1 = pd.read_csv('Prophet_Brezeln.csv')

df_p2 = pd.read_csv('Prophet_Laugenstange.csv')

df_p3 = pd.read_csv('Prophet_Laugenbroetchen.csv')

In [ ]:
#"Sales" Szenarien -> Key für die Python Dictionaries
scenarios_p1 = list(df_p1['Sc'])
scenarios_p2 = list(df_p2['Sc'])
scenarios_p3 = list(df_p3['Sc'])

#Für ausgewählte Tage:
demand_p1 = dict(zip(df_p1.Sc, df_p1['18078']))
prob_p1 = dict(zip(df_p1.Sc, df_p1.prob))

demand_p2 = dict(zip(df_p2, df_p2['18078']))
prob_p2 = dict(zip(df_p2_new.Sc, df_p2_new.prob))

demand_p3 = dict(zip(df_p3.Sc, df_p3['18078']))
prob_p3 = dict(zip(df_p3.Sc, df_p3.prob))


#Preise & Kosten (anonymisiert)
price_p1 = 1
costs_p1 = 1
price_p2 = 1
costs_p2 = 1
price_p3 = 1
costs_p3 = 1


#Parameter werden für jedes Szenario angepasst
scvl = 0.8

alpha_12 = 1
alpha_13 = 1

alpha_21 = 1
alpha_23 = 1

alpha_31 = 1
alpha_32 = 1

In [ ]:
m = Model()

#Dummy Variable für M-Methode
M1 = m.addVars(scenarios_p1, vtype=GRB.BINARY, name = "M1")
M2 = m.addVars(scenarios_p2, vtype=GRB.BINARY, name = "M2")
M3 = m.addVars(scenarios_p3, vtype=GRB.BINARY, name = "M3")

#Produkt 1 
order_p1 = m.addVar(vtype=GRB.INTEGER, name = "order_p1")
sales_p1 = m.addVars(scenarios_p1,vtype=GRB.INTEGER, name = "sales_p1")
sub_p12 = m.addVars(scenarios_p2, vtype=GRB.INTEGER, name = "sub_p12")
sub_p13 = m.addVars(scenarios_p3, vtype=GRB.INTEGER, name = "sub_p13")

#Produkt 2
order_p2 = m.addVar(vtype=GRB.INTEGER, name = "order_p2")
sales_p2 = m.addVars(scenarios_p2, vtype=GRB.INTEGER, name = "sales_p2")
sub_p21 = m.addVars(scenarios_p1, vtype=GRB.INTEGER, name = "sub_p21")
sub_p23 = m.addVars(scenarios_p3, vtype=GRB.INTEGER, name = "sub_p23")

#Produkt 3
order_p3 = m.addVar(vtype=GRB.INTEGER, name = "order_p3")
sales_p3 = m.addVars(scenarios_p3, vtype=GRB.INTEGER, name = "sales_p3")
sub_p31 = m.addVars(scenarios_p1, vtype=GRB.INTEGER, name = "sub_p31")
sub_p32 = m.addVars(scenarios_p2, vtype=GRB.INTEGER, name = "sub_p32")

#Sub Variable ist so zu lesen: Sub_p32 = Substitution von Produkt 2 zu Produkt 3

#Sales Constraint
m.addConstrs((sales_p1[s] <= order_p1 for s in scenarios_p1), name = "Cons1")
m.addConstrs((sales_p2[s] <= order_p2 for s in scenarios_p2), name = "Cons2")
m.addConstrs((sales_p3[s] <= order_p3 for s in scenarios_p3), name = "Cons3")

#Demand Constraint
m.addConstrs((sales_p1[s] <= demand_p1[s] for s in scenarios_p1), name = "Cons4")
m.addConstrs((sales_p2[s] <= demand_p2[s] for s in scenarios_p2), name = "Cons5")
m.addConstrs((sales_p3[s] <= demand_p3[s] for s in scenarios_p3), name = "Cons6")

#Service Level Constraint
m.addConstr((order_p1 >= quicksum(demand_p1[s]*prob_p1[s] for s in scenarios_p1)* scvl), name = "Cons101")
m.addConstr((order_p2 >= quicksum(demand_p2[s]*prob_p2[s] for s in scenarios_p2)* scvl), name = "Cons102")
m.addConstr((order_p3 >= quicksum(demand_p3[s]*prob_p3[s] for s in scenarios_p3)* scvl), name = "Cons103")

#Sales & Substitution constraint
m.addConstrs(( (sub_p12[m] + sub_p13[p] + sales_p1[s]) <= order_p1 for p in scenarios_p3 for m in scenarios_p2 for s in scenarios_p1), name = "Cons7")
m.addConstrs(( (sub_p21[s] + sub_p23[p] + sales_p2[m]) <= order_p2 for p in scenarios_p3 for s in scenarios_p1 for m in scenarios_p2), name = "Cons8")
m.addConstrs(( (sub_p31[s] + sub_p32[m] + sales_p3[p]) <= order_p3 for m in scenarios_p2 for s in scenarios_p1 for p in scenarios_p3), name = "Cons9")


#M-Methode für jedes Produkt
m.addConstrs((sales_p1[s] >= order_p1 - 100000 * (1-M1[s]) for s in scenarios_p1), name = "ConsB1")
m.addConstrs((sales_p1[s] + 0.000001 <= order_p1 + 100000 * M1[s] for s in scenarios_p1), name = "ConsB2")
m.addConstrs((sub_p21[s] == M1[s]*(demand_p1[s]-order_p1) for s in scenarios_p1), name = "ConsB3")
m.addConstrs((sub_p31[s] == M1[s]*(demand_p1[s]-order_p1) for s in scenarios_p1), name = "ConsB3")

m.addConstrs((sales_p2[s] >= order_p2 - 100000 * (1-M2[s]) for s in scenarios_p2), name = "ConsB4")
m.addConstrs((sales_p2[s] + 0.000001 <= order_p2 + 100000 * M2[s] for s in scenarios_p2), name = "ConsB5")
m.addConstrs((sub_p12[s] == M2[s]*(demand_p2[s]-order_p2) for s in scenarios_p2), name = "ConsB6") 
m.addConstrs((sub_p32[s] == M2[s]*(demand_p2[s]-order_p2) for s in scenarios_p2), name = "ConsB7")

m.addConstrs((sales_p3[s] >= order_p3 - 100000 * (1-M3[s]) for s in scenarios_p3), name = "ConsB8")
m.addConstrs((sales_p3[s] + 0.000001 <= order_p3 + 100000 * M3[s] for s in scenarios_p3), name = "ConsB9")
m.addConstrs((sub_p23[s] == M3[s]*(demand_p3[s]-order_p3) for s in scenarios_p3), name = "ConsB10") 
m.addConstrs((sub_p13[s] == M3[s]*(demand_p3[s]-order_p3) for s in scenarios_p3), name = "ConsB11")


In [ ]:
m.update()

#Objective Function
obj = quicksum(
            (prob_p1[s]*sales_p1[s]*price_p1 + prob_p2[m] * alpha_12 * price_p1 * sub_p12[m] + prob_p3[p] * alpha_13 * price_p1 * sub_p13[p] - prob_p1[s]*(order_p1 * costs_p1)  + 
            prob_p2[m]*sales_p2[m]*price_p2 + prob_p1[s] * alpha_21 * price_p2 * sub_p21[s] + prob_p3[p] * alpha_23 * price_p2 * sub_p23[p] - prob_p2[m]*(order_p2 * costs_p2) +
            prob_p3[p]*sales_p3[p]*price_p3 + prob_p1[s] * alpha_31 * price_p3 * sub_p31[s] + prob_p2[m] * alpha_32 * price_p3 * sub_p32[m] - prob_p3[p]*(order_p3 * costs_p3) for s in scenarios_p1 for m in scenarios_p2 for p in scenarios_p3)
      ) 

#Reduzierung der Parameter damit RAM nicht voll läuft
m.Params.DualReductions = 0
m.Params.Threads = 1
m.Params.NodefileStart = 0.5

m.setObjective(obj, GRB.MAXIMIZE)

m.optimize()
m.printAttr('X')